In [246]:
import numpy as np
from hashlib import sha256, sha1
from numpy.random import randint
from math import ceil

# Proof of Work Simulation

- PoW = randomly seraching for a number between 0 and 100 (very simplified)
- Each block can contain 4 transactions
- There are 4 users initialized in the simulation
- Bad miners will try to add fake transactions to the chain

In [277]:
# Parameters
NONCE_LOW = 0
NONCE_HIGH = 500

class Blockchain():
    """A collection of Block objects linked together through the hash
    of the previous block in the chain.
    """
    
    def __init__(self, head):
        self.head = head
        
    def addBlock(self, block):
        block.next = self.head
        block.hash = block.goalHash()
        for transaction in block.data:
            transaction.make()
        self.head = block
    
    def size(self):
        temp = self.head
        nextBlock = temp.nextBlock
        k = 0
        while nextBlock:
            temp = nextBlock
            nextBlock = nextBlock.nextBlock
            k += 1
        return k
    
    def latestBlock(self):
        return self.head

def packageTransaction(transactionList):
    """Takes in a list of transactions (max of 4 in a block) and
    formats it into a string that can be used to calculate the
    hash of a block.
    """
    formatted = ''
    for transaction in transactionList:
        formatted += str(transaction) + ';'
    return formatted

class Block():
    """An individual block that contains a signed hash (that fulfills
    certain criteria), data in the form of transactions, and the previous
    block's hash. Once a miner finds a suitable signed hash, a block
    is created and the miner is rewarded tokens.
    """    
    def __init__(self, index, previousBlock, data, nonce):
        self.previousBlock = previousBlock
        self.index = index        
        self.hash = None
        
        if previousBlock and data and nonce:
            self.data = data
            self.nonce = nonce
            self.previousBlock = previousBlock
        else:
            self.hash = sha256('0'.encode()).hexdigest()
        
        self.signedBy = None # whoever finds the solution first signs
        
        self.possibleNextBlocks = []
        self.nextBlock = None
    
    def sign(self, miner):
        self.signedBy = miner
        miner.blocksSigned += 1
        miner.reward += 12.5
        
    # This is for this mini example only, not actually what happens.
    def goalHash(self):            
        formattedTx = packageTransaction(self.data)
        # Nonce + Transaction Data + Previous Hash
        goalHash = str(self.nonce) + formattedTx + self.previousBlock.hash
        return makeHash(goalHash)
    
class User():
    """Someone who is making transactions in the network.
    """
    
    def __init__(self, userId):
        self.userId = userId
        self.balance = 100 # Initialize everyone to have some starting balance
        
    def pay(self, other, amount):
        self.balance -= amount
        other.balance += amount
        
    def __str__(self):
        return 'User #{} has a balance of {}.'.format(self.userId, self.balance) 
    
class Miner():
    """A miner 
    """
    def __init__(self, minerId):
        self.minerId = minerId
        self.reward = 0
        self.blocksSigned = 0
        
    def mine(self, block):
        randNum = randint(NONCE_LOW, NONCE_HIGH)
        guess, k = randint(NONCE_LOW, NONCE_HIGH), 0
        guessHash = ''
        goalHash = block.goalHash()
        while guessHash != goalHash:
            guess = randint(NONCE_LOW, NONCE_HIGH)
            hashStr = str(guess) + packageTransaction(block.data) + block.previousBlock.hash
            guessHash = makeHash(hashStr)
            k += 1 # Incrememnt the number of computations used          
        return k
    
    def validate(self, blockTransaction, ledgerTransaction):
        if len(blockTransaction) != len(ledgerTransaction):
            return False
        for i in range(len(blockTransaction)):
            if blockTransaction[i] != ledgerTransaction[i]:
                return False
        return True
            
    def __str__(self):
        return 'Miner #' + str(self.minerId)

class BadMiner(Miner):
    """A nefarious miner who wants to mutate the chain to create
    fake/false transactions.
    """ 
    def validate(self, blockTransaction, ledgerTransaction):
        return not super(BadMiner, self).validate(blockTransaction, ledgerTransaction)
    
    def __str__(self):
        return 'Bad Miner #' + str(self.minerId) 
        
class Transaction():
    def __init__(self, fromUser, toUser, amount):        
        # user1 -- amount -> user2
        self.fromUser = fromUser
        self.toUser = toUser
        self.amount = amount
        self.valid = (fromUser.balance - amount) > 0
        
    def __str__(self):
        return 'User ' + str(self.fromUser.userId) + ' sends ' + str(self.amount) + ' to User ' + str(self.toUser.userId)
    
    def __eq__(self, other):
        return self.fromUser.userId == other.fromUser.userId and self.toUser.userId == other.toUser.userId and self.amount == other.amount
    
    def __ne__(self, other):
        return not self.__eq__(other)
    
    def make(self):
        self.fromUser.pay(self.toUser, self.amount)
    
def makeHash(s, f=sha256):
    return f(s.encode()).hexdigest()

In [289]:
# Create network of miners
networkSize = 100

# Create test users
numUsers = 4
user0, user1, user2, user3 = [User(i) for i in range(numUsers)]

# Initialize blockchain
blockchain = Blockchain(Block(0, None, None, None))

minerNetwork = []

# Assume blocks can only hold 4 transactions
# Test for 3 blocks
testTransactions = [
    [Transaction(user0, user1, 20), 
     Transaction(user2, user3, 30), 
     Transaction(user3, user1, 3), 
     Transaction(user3, user0, 15)],
    
    [Transaction(user1, user0, 5), 
     Transaction(user3, user0, 2), 
     Transaction(user1, user2, 3), 
     Transaction(user2, user0, 30)],
    
    [Transaction(user3, user0, 60), 
     Transaction(user2, user3, 30), 
     Transaction(user3, user1, 1), 
     Transaction(user0, user3, 12)]
]

fakeTransaction = [
    Transaction(user1, user0, 100), 
    Transaction(user2, user0, 100), 
    Transaction(user3, user0, 50), 
    Transaction(user3, user0, 50)
]

def runSimulation(percentBad=0):
    goodMiners = int(networkSize * (1 - percentBad))
    
    # Create miner network with x good miners and networkSize - x bad miners
    global minerNetwork
    minerNetwork = [Miner(i) for i in range(goodMiners)] + [BadMiner(i) for i in range(goodMiners, networkSize)]
    
    computations = []
    neededComputations = []
    for i in range(len(testTransactions)):
        
        print('Block #{}'.format(i + 1))
        
        fakeNonce = randint(NONCE_LOW, NONCE_HIGH)
        realBlock = Block(blockchain.size(), blockchain.latestBlock(), testTransactions[i], fakeNonce)
        fakeBlock = Block(blockchain.size(), blockchain.latestBlock(), fakeTransaction, fakeNonce)
        
        minerComputations = []
        for miner in minerNetwork:
            newBlock = realBlock
            if isinstance(miner, BadMiner):
                newBlock = fakeBlock

            if newBlock not in blockchain.latestBlock().possibleNextBlocks:
                blockchain.latestBlock().possibleNextBlocks.append(newBlock)
            minerComputations.append(miner.mine(newBlock))
        
        # Find out the number of wasted computations
        totalComputations = sum(minerComputations)
        print('Number of total computations = {}'.format(totalComputations))
        computations.append(totalComputations)
        
        # Choose the winning miner based on the miner that takes the least number
        # of computations to get a valid hash.
        winningMiner = minerNetwork[np.argmin(minerComputations)]
        neededComputations.append(min(minerComputations))
        
        # This miner gets to sign off the new block, and is rewarded with tokens.
        newBlock.sign(winningMiner)
            
        for possibleBlock in blockchain.latestBlock().possibleNextBlocks:
            
            validation = [miner.validate(possibleBlock.data, testTransactions[i]) for miner in minerNetwork]
            percentValidation = np.mean(validation)
                                
            if percentValidation > 0.5:
                blockchain.addBlock(possibleBlock)
                if possibleBlock is fakeBlock:
                    print("Fake Block Validation: ", percentValidation)
                    print("Real Block Validation: ", 1 - percentValidation)
                if possibleBlock is realBlock:
                    print("Real Block Validation: ", percentValidation)
                    print("Fake Block Validation: ", 1 - percentValidation)
                        
                for data in blockchain.latestBlock().data:
                    print(data)
                if possibleBlock is realBlock:
                    print('Majority rules. Main chain is safe!')
                elif possibleBlock is fakeBlock:
                    print('51% attack has occurred. Main chain now contains the fake transactions!')
        print('\n')
        
    avgTotal = np.mean(computations)
    avgNeeded = np.mean(neededComputations)
    percentWasted = (avgTotal - avgNeeded) / avgTotal
    print('Average number of computations = {}'.format(avgTotal))
    print('Average number of computations used by the block miners = {}'.format(avgNeeded))
    print('Percent computation wasted = {}\n'.format(percentWasted))

## All Miners are Good

In this example, all miners in the network are set to be "good," which means that the each block will be validated by 100% of miners. *Pay attention to the amount of computation that is wasted.*

In [286]:
# All miners are good
# Run above cell before any simulation to reset the environment
runSimulation()
print('Final User Balances:')
for user in [user0, user1, user2, user3]:
    print(user)
    
print('\n\nMiner Rewards:')
for miner in minerNetwork:
    print(miner, miner.reward)

Block #1
Number of total computations = 37963
Real Block Validation:  1.0
Fake Block Validation:  0.0
User 0 sends 20 to User 1
User 2 sends 30 to User 3
User 3 sends 3 to User 1
User 3 sends 15 to User 0
Majority rules. Main chain is safe!


Block #2
Number of total computations = 54046
Real Block Validation:  1.0
Fake Block Validation:  0.0
User 1 sends 5 to User 0
User 3 sends 2 to User 0
User 1 sends 3 to User 2
User 2 sends 30 to User 0
Majority rules. Main chain is safe!


Block #3
Number of total computations = 44424
Real Block Validation:  1.0
Fake Block Validation:  0.0
User 3 sends 60 to User 0
User 2 sends 30 to User 3
User 3 sends 1 to User 1
User 0 sends 12 to User 3
Majority rules. Main chain is safe!


Average number of computations = 45477.666666666664
Average number of computations used by the block miners = 6.666666666666667
Percent computation wasted = 0.9998534078998483

Final User Balances:
User #0 has a balance of 180.
User #1 has a balance of 116.
User #2 has a b

## One Bad Miner

When there is <= 50% of attackers in the network, then proof of work will do what it's supposed to: protect the immutability of the chain from attackers. Attackers are disincentivized to try to mutate the chain with fake transactions, since without more than 51% control of the network, the attackers will simply be wasting compute that could be used for actually mining blocks properly.

In [288]:
# One bad miner
runSimulation(percentBad=0.01)
print('Final User Balances:')
for user in [user0, user1, user2, user3]:
    print(user)

Block #1
Number of total computations = 55248
Real Block Validation:  0.99
Fake Block Validation:  0.010000000000000009
User 0 sends 20 to User 1
User 2 sends 30 to User 3
User 3 sends 3 to User 1
User 3 sends 15 to User 0
Majority rules. Main chain is safe!


Block #2
Number of total computations = 46665
Real Block Validation:  0.99
Fake Block Validation:  0.010000000000000009
User 1 sends 5 to User 0
User 3 sends 2 to User 0
User 1 sends 3 to User 2
User 2 sends 30 to User 0
Majority rules. Main chain is safe!


Block #3
Number of total computations = 55777
Real Block Validation:  0.99
Fake Block Validation:  0.010000000000000009
User 3 sends 60 to User 0
User 2 sends 30 to User 3
User 3 sends 1 to User 1
User 0 sends 12 to User 3
Majority rules. Main chain is safe!


Average number of computations = 52563.333333333336
Average number of computations used by the block miners = 9.666666666666666
Percent computation wasted = 0.999816094869681

Final User Balances:
User #0 has a balance 

## 51% Attack

When 51% of the miners in the network are "bad miners," then they can override the good miners by majority vote by validating their own false transactions. Thus, the fake block generated by the network attackers is validated by 51% of the network, which is a majority.

In [290]:
# 51% attack
runSimulation(percentBad=0.51)    
print('Oh no! User 0 gets all the crypto! :(\n')

print('Final User Balances:')
for user in [user0, user1, user2, user3]:
    print(user)

Block #1
Number of total computations = 54053
Fake Block Validation:  0.51
Real Block Validation:  0.49
User 1 sends 100 to User 0
User 2 sends 100 to User 0
User 3 sends 50 to User 0
User 3 sends 50 to User 0
51% attack has occurred. Main chain now contains the fake transactions!


Block #2
Number of total computations = 55542
Fake Block Validation:  0.51
Real Block Validation:  0.49
User 1 sends 100 to User 0
User 2 sends 100 to User 0
User 3 sends 50 to User 0
User 3 sends 50 to User 0
51% attack has occurred. Main chain now contains the fake transactions!


Block #3
Number of total computations = 51561
Fake Block Validation:  0.51
Real Block Validation:  0.49
User 1 sends 100 to User 0
User 2 sends 100 to User 0
User 3 sends 50 to User 0
User 3 sends 50 to User 0
51% attack has occurred. Main chain now contains the fake transactions!


Average number of computations = 53718.666666666664
Average number of computations used by the block miners = 4.333333333333333
Percent computation 